## This nothebook shows how to compute the compression and time ratio for a given quantized network. Moreover, it contains how to create a PathoNet model using the Index Map method and how to load the created compressed model.

In [1]:
import tensorflow as tf
import pickle
import numpy as np
import imageio
import json
from scipy import misc
import timeit
import math
import os
from time_space_eval_pathonet import make_submodel, \
        extract_conv_info, extract_compressed_weights, \
        read_labels, space_calc, product_time, \
        residualDilatedInceptionModule_iMap, PathoNet_iMaps, \
        utils_for_Patho_iMap

In [2]:
# Avoid printing INFO, WARNING, and ERROR messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [3]:
#Activate GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
    # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# Load original model
model = tf.keras.models.load_model("./original_nets/PathoNet.hdf5")

In [5]:
# Change this path to use a different compressed model

path_quant="./pathonet_compressed_models/experiments/CWS_k256/0-0-0-256-0.0001-1e-05-5-75-147.1456.h5"

In [6]:
# Load the considered "quantized" model and set weights in original model structure
with open(path_quant, "rb") as weights:
    lw = pickle.load(weights)
model.set_weights(lw)

In [7]:
#Load image for testing
imageShape = [1228, 1228, 3]
    
inputShape = [256, 256, 3]

### The next two cells show how to compute the __compression ratio__ and __time ratio__ for a quantized network

In [8]:
# Slow down (used just 3 images from the test set to make it faster)
testset = "./SHIDC-B-Ki-67/Test"
data = [testset+"/"+f for f in os.listdir(testset) if '.jpg' in f][:3]
original, quant = product_time(model, data)
print(round(quant/original, 3))

0.009861061960691586 0.00986462298897095
0.03545475995633751 0.03537583598517813
0.04716759594157338 0.047429439990082756
0.06286244696821086 0.06419499800540507
0.07872045200201683 0.0805153010005597
0.09412119004991837 0.09618389001116157
0.10378173107164912 0.10682839201763272
0.11782630806555972 0.12344232003670186
0.13042085405322723 0.13859099106048234
0.1435904960380867 0.1540073030628264
0.15326975504285656 0.16664277706877328
0.1636524120112881 0.18144620806560852
0.17738681298214942 0.20291017007548362
0.19037975100218318 0.22306630510138348
0.20419614802813157 0.2514092721394263
0.21960680303163826 0.2810178110958077
0.24025575703126378 0.32665589309181087
0.2616521820309572 0.37509737408254296
0.2735376709897537 0.3895633890933823
0.2815710559953004 0.3992237210914027
0.2948036270099692 0.4197049791109748
0.3078800270159263 0.4398326871159952
0.32155526903807186 0.46014415510580875
0.3347235830442514 0.4804683341062628
0.35874816702562384 0.506286231073318
0.370814204012276

In [9]:
# Compute space compression
space_original, space_compressed = space_calc(model)

print(round(space_compressed / space_original, 3))

0.254


### Following, the creation of the PathoNet iMap model and how to load it

In [10]:
from tensorflow.keras.layers import (Input,Add,add,concatenate,Activation,concatenate,
                        Concatenate,Dropout,BatchNormalization,Reshape,Permute,
                        Dense,UpSampling2D,Flatten,Lambda,Activation,Conv2D,
                        DepthwiseConv2D,ZeroPadding2D,GlobalAveragePooling2D,
                        MaxPooling2D,AveragePooling2D,LeakyReLU,Conv2DTranspose)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import get_source_inputs
from tensorflow.keras.activations import relu
from tensorflow.keras.optimizers import SGD, Adam

In [11]:
#Original Code of PathoNet

def residualDilatedInceptionModule(y, nb_channels, _strides=(1, 1),t="e"):
    if t=="d":
        y = Conv2D(nb_channels, kernel_size=(1, 1), strides=(1, 1),kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)
        y = Conv2D(nb_channels, kernel_size=(1, 1), strides=(1, 1),kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
        y = BatchNormalization()(y)
        y = LeakyReLU()(y)


    A1 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(y)
    A1 = BatchNormalization()(A1)
    A1 = LeakyReLU()(A1)
    A1 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), padding='same', use_bias=False)(A1)
    A1 = BatchNormalization()(A1)
    A1 = LeakyReLU()(A1)


    A4 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4),  dilation_rate=4, padding='same', use_bias=False)(y)
    A4 = BatchNormalization()(A4)
    A4 = LeakyReLU()(A4)
    A4 = Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides,kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4),  dilation_rate=4, padding='same', use_bias=False)(A4)
    A4 = BatchNormalization()(A4)
    A4 = LeakyReLU()(A4)

    if (t=="e"):
        y=concatenate([y,y])
    y=add([A1,A4,y])
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)

    return y


def PathoNet(input_size = (256,256,3), classes=3, pretrained_weights = None):
    inputs = Input(input_size) 

    block1= Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(inputs)
    block1 = BatchNormalization()(block1)
    block1 = LeakyReLU()(block1)
    block1= Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block1)
    block1 = BatchNormalization()(block1)
    block1 = LeakyReLU()(block1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(block1)

    block2= residualDilatedInceptionModule(pool1,32,t="e")
    pool2 = MaxPooling2D(pool_size=(2, 2))(block2)

    block3= residualDilatedInceptionModule(pool2,64,t="e")
    pool3 = MaxPooling2D(pool_size=(2, 2))(block3)

    block4= residualDilatedInceptionModule(pool3,128,t="e")
    pool4 = MaxPooling2D(pool_size=(2, 2))(block4)
    drop4 = Dropout(0.1)(pool4)

    block5= residualDilatedInceptionModule(drop4,256,t="e")
    drop5 = Dropout(0.1)(block5)

    up6 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(drop5)),128,t="d")
    merge6 = concatenate([block4,up6], axis = 3)

    up7 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge6)),64,t="d")
    merge7 = concatenate([block3,up7], axis = 3)

    up8 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge7)),32,t="d")
    merge8 = concatenate([block2,up8], axis = 3)

    up9 = residualDilatedInceptionModule((UpSampling2D(size = (2,2))(merge8)),16,t="d")
    merge9 = concatenate([block1,up9], axis = 3)

    block9=Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(merge9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    block9=Conv2D(16, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    block9=Conv2D(8, 3, padding = 'same', kernel_initializer = 'orthogonal',kernel_regularizer= l2(5e-4), use_bias=False)(block9)
    block9 = BatchNormalization()(block9)
    block9 = LeakyReLU()(block9)
    conv10 = Conv2D(classes, 1, activation = 'relu')(block9)

    model = tf.keras.models.Model(inputs = inputs, outputs = conv10)

    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
    

    
    return model

In [12]:
#Reload weights and set on model created as new

with open(path_quant, "rb") as weights:
    lw = pickle.load(weights)

patho = PathoNet(input_size = (256,256,3), classes=3)
patho.set_weights(lw)

In [13]:
#Create PathoNet with iMap

res_weights, last_bias, indexes_weights, vect_centers = utils_for_Patho_iMap(patho)
patho_imap = PathoNet_iMaps(last_bias, indexes_weights, vect_centers, res_weights = res_weights)

In [14]:
#Check predict between original model and iMap model
d= data[1]
img=imageio.imread(d)
labels=read_labels(d.replace(".jpg",".json"),inputShape,imageShape).reshape((-1,3))
img=np.expand_dims(np.array(misc.imresize(img,size=(inputShape[0], inputShape[1])))/255, axis=0)
np.all(np.isclose(patho_imap(img), patho(img), rtol=1e-05, atol=1e-04))

/home/jessica/envs/sham/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.
  """


True

In [15]:
#Check with save model to h5 and load

patho_imap.save("patho_imap.h5")
loaded_model = tf.keras.models.load_model("patho_imap.h5")

np.all(np.isclose(loaded_model(img), patho(img), rtol=1e-05, atol=1e-04))

True

In [ ]:
import lzma
with lzma.open("res_weights.xz", "wb") as f:
    pickle.dump(res_weights, f)

with lzma.open("last_bias.xz", "wb") as f:
    pickle.dump(last_bias, f)

with lzma.open("indexes_weights.xz", "wb") as f:
    pickle.dump(indexes_weights, f)

with lzma.open("vect_centers.xz", "wb") as f:
    pickle.dump(vect_centers, f)

    
    

with lzma.open("res_weights.xz", "rb") as f:
    res_weights= pickle.load(f)


with lzma.open("last_bias.xz", "rb") as f:
    last_bias = pickle.load(f)


with lzma.open("indexes_weights.xz", "rb") as f:
    indexes_weights = pickle.load(f)


with lzma.open("vect_centers.xz", "rb") as f:
    vect_centers = pickle.load(f)
    

    
patho_imap = PathoNet_iMaps(last_bias, indexes_weights, vect_centers, res_weights = res_weights)
